In [1]:
# current conda base: base 
import anndata
import numpy as np
from remote_conda_decorator import wrap, SmartAnnData

# 1. 模拟生成一个 adata (在当前环境)
# 实际场景中你可能已经 load 了一个巨大的 adata
adata = anndata.AnnData(np.random.rand(100, 100))
adata.obs['group'] = 'control'
print(f"Local adata shape: {adata.shape}")

# 2. 定义远程函数
# 注意参数名为 smart_obj
@wrap(base='scOmni')
def analyze_cell(smart_obj, n_neighbors=15):
    import scanpy as sc
    import squidpy as sq 
    print(sq.__version__)
    
    # 获取真正的 adata 对象
    remote_adata = smart_obj.adata
    print(f"Remote received adata: {remote_adata.shape}")
    # 执行计算 (在 scOmni 环境中)
    # sc.pp.pca(remote_adata)
    # sc.pp.neighbors(remote_adata, n_neighbors=n_neighbors)
    # sc.tl.umap(remote_adata)
    # 我们可以返回处理后的 adata，包装在 SmartAnnData 中以快速传回
    remote_adata = sq.datasets.seqfish()
    return SmartAnnData(remote_adata)

# 3. 调用
# 关键步骤：使用 SmartAnnData 包装你的 adata
smart_input = SmartAnnData(adata)

print("--- Starting Remote Execution ---")
# 此时，adata 会被瞬间写入 /dev/shm (内存)，几乎不耗时
result_adata = analyze_cell(smart_input, n_neighbors=30)

print("--- Execution Finished ---")
print(f"Result contains UMAP: {'X_umap' in result_adata.obsm}")

Local adata shape: (100, 100)
--- Starting Remote Execution ---
⚡ [SmartAnnData] Saving AnnData to RAM disk: /dev/shm/adata_55efb0c6-b5d0-42d5-8974-3e4982fce34d.h5ad ...
[scOmni] ⚡ [SmartAnnData] Loaded AnnData from RAM disk in remote env.
[scOmni] 1.6.5
[scOmni] Remote received adata: (100, 100)
[scOmni] ⚡ [SmartAnnData] Saving AnnData to RAM disk: /dev/shm/adata_d7f08775-1541-4359-9b2f-b84f6d4bd213.h5ad ...
⚡ [SmartAnnData] Loaded AnnData from RAM disk in remote env.
🧹 [SmartAnnData] Cleaned up /dev/shm/adata_55efb0c6-b5d0-42d5-8974-3e4982fce34d.h5ad
--- Execution Finished ---
Result contains UMAP: True


In [2]:
result_adata

AnnData object with n_obs × n_vars = 19416 × 351
    obs: 'Area', 'celltype_mapped_refined'
    uns: 'celltype_mapped_refined_colors'
    obsm: 'X_umap', 'spatial'